In [5]:
import IPython.display as ipd
import numpy as np
import pandas as pd
import h5py

In [ ]:
from LibriSpeech_utt import *

path = '/fs/scratch/PAS0774/Shanmukh/Databases/LibriSpeech/LibriSpeech/train-clean-100'
T = 1.6
libri_dataset = LibriSpeechUttDataset(path, T, return_vad=True)
print(f"Dataset Size: {len(libri_dataset)}")
#breakpoint()

train_loader = DataLoader(libri_dataset, batch_size = 10, num_workers=0)

for _batch_idx, val in enumerate(train_loader):
    print(f"sig: {val[0].shape}, {val[0].dtype}, {val[0].device}")
    if _batch_idx==0:
        break

In [ ]:
x = libri_dataset[5]
ipd.Audio(x[0], rate=16000, autoplay=True)

In [ ]:
# reading FSDNoisy_18k

from FSDNoisy18k import *

path = '/fs/scratch/PAS0774/Shanmukh/Databases/FSDnoisy18k.audio_train'
T = 1.6
noise_dataset = FSDNoisy18kUttDataset(path, T, return_vad=True)
print(f"Dataset Size: {len(noise_dataset)}")

train_loader = DataLoader(noise_dataset, batch_size = 1, num_workers=0)

for _batch_idx, val in enumerate(train_loader):
    print(f"sig: {val[0].shape}, {val[0].dtype}, {val[0].device}")
    if _batch_idx==0:
        break
	

In [ ]:
x = noise_dataset[5]

In [ ]:
x.shape

In [ ]:
ipd.Audio(x[0,:].numpy(), rate=16000, autoplay=True)

In [ ]:
from LibriSpeech_utt import *

path = '/fs/scratch/PAS0774/Shanmukh/Databases/LibriSpeech/LibriSpeech/dev-clean'
T = 1.6
libri_dev_dataset = LibriSpeechUttDataset(path, T, return_vad=True)

In [ ]:
snr = np.random.uniform(-6, 6, size=5000)

spk_indices = np.random.randint(0,len(libri_dev_dataset), size=(5000, 1))
noise_indices = np.random.randint(0,len(noise_dataset), size=(5000, 1))

spk_files, noise_files = [],[]
for idx in range(0, spk_indices.shape[0]):
    spk_files.append(libri_dev_dataset.file_list[spk_indices[idx,0]])
    noise_files.append(noise_dataset.file_list[noise_indices[idx,0]])
    

In [ ]:
rirs_path=[]
for _idx in range(5000):
    idx_folder = (_idx//5000)*5000
    rirs_path.append(f"/fs/scratch/PAA0005/Shanmukh/Habets_SignalAware_Doa/RIRs/Validation/Rirs_{idx_folder}/rirs_{_idx}.h5")

In [ ]:
mixing_dict = {
    "spk": spk_files, 
    "noi": noise_files, 
    "snr": snr, 
    "rirs": rirs_path
}

In [ ]:
mixing_df = pd.DataFrame(mixing_dict)
mixing_df.to_csv("validation_files_list.csv", index=False)

In [ ]:
mixing_df.iloc[0]

In [ ]:
import torch
v = torch.normal(0, 1, [4, 25600])

In [ ]:
a = -10 * torch.rand(1) + 30

In [ ]:
10**(a/10)

In [ ]:
f = h5py.File("../Logs/tr_ex_1.h5", 'r')

mix_sph = np.array(f["mix_sph"])

dp_sph = np.array(f["spk"]["dp_signal"])

In [ ]:
mix_sph.dtype

In [ ]:
ipd.Audio(mix_sph[0,:], rate=16000, autoplay=True)

In [ ]:
ipd.Audio(dp_sph[0,:], rate=16000, autoplay=True)

In [ ]:
ax = str(np.array(f["ex_gen_cfg"]["rirs"]))

In [ ]:
ax

In [ ]:
np.array(f["ex_gen_cfg"]["snr"])

In [ ]:
#list of files

path = "/fs/scratch/PAA0005/Shanmukh/Habets_SignalAware_Doa/Signals/Train/**/*.h5"

In [ ]:
from glob import glob

files_list = glob(path)

In [ ]:
import re
sorted(files_list, key=lambda s: int(re.search(r'\d+', s[-8:]).group()))


In [ ]:
#dbg
import os
import glob
import re
import torch
from network_input_output import *
from torch.utils.data import Dataset, DataLoader
import h5py
import numpy as np
from dataset import *

root_path = "/fs/scratch/PAA0005/Shanmukh/Habets_SignalAware_Doa/Signals/Validation_correction/"
transforms = [SignalAwareDoA_features(frame_len=512, frame_shift=256, doa_resolution = 5, array_type=None, array_setup=None, fs=16000)]
train_dataset = SignalAwareDoADataset(root_path, transforms)

In [ ]:
train_dl = DataLoader(train_dataset, batch_size=16, num_workers=10)

for idx, ex in enumerate(train_dl):
    mix_sph_mag, mix_sph_ph, tgt_spk_cs, spk_doas = ex
    #print(f"{mix_sph_mag.shape}, {mix_sph_ph.shape}, {tgt_spk_cs.shape}, {spk_doas}")
    #print(f"{mix_sph_mag.dtype}, {mix_sph_ph.dtype}, {tgt_spk_cs.dtype}, {spk_doas.dtype}")
    if torch.isnan(mix_sph_mag).any():
        print(f"idx: {idx}, {torch.where(torch.isnan(mix_sph_mag))}")
    else:
        print(f"idx: {idx} .....")

In [ ]:
b_idx = 5
ex_idx = 14
mix_sph_mag, mix_sph_ph, tgt_spk_cs, spk_doas = train_dataset[16*b_idx + ex_idx]
print(f"{torch.where(torch.isnan(mix_sph_mag))}")

In [ ]:
#output: torch signal
def compute_vad(source_signal: 'Array ( sig_len)', frame_size: 'int (samples) ', frame_shift: 'int (samples)', fs: int = 16000):
    import webrtcvad
    vad = webrtcvad.Vad()
    agressiveness=3
    vad.set_mode(agressiveness)

    #sig_vad = np.zeros_like(source_signal)
    sig_vad = []
    sig_size = source_signal.shape[-1]

    for frame_idx, frm_strt in enumerate(range(0, sig_size-frame_size +1, frame_shift)):
        frame = source_signal[frm_strt:frm_strt + frame_size] #source_signal[frame_idx * vad_frame_len: (frame_idx + 1) * vad_frame_len]
        frame_bytes = (frame * 32767).astype('int16').tobytes()
        #sig_vad[frm_strt:frm_strt + frame_size] = vad.is_speech(frame_bytes, fs)
        sig_vad.append(vad.is_speech(frame_bytes, fs))

    return torch.tensor(np.array(sig_vad))

# Dataset debugging
1. Compute Fraction of speech frames in the training utterances: Histogram plot
2. DOA histogram plot of training example

In [ ]:
#dbg
import os
import glob
import re
import torch
from network_input_output import *
from torch.utils.data import Dataset, DataLoader
import h5py
import numpy as np
from dataset import *

root_path = "/fs/scratch/PAA0005/Shanmukh/Habets_SignalAware_Doa/Signals/Train_correction/"
transforms = [SignalAwareDoA_features(frame_len=512, frame_shift=256, doa_resolution = 5, array_type=None, array_setup=None, fs=16000)]
train_dataset = SignalAwareDoADataset(root_path, transforms=None)

train_dl = DataLoader(train_dataset, batch_size=1, num_workers=1)
sph_frm_perc_list = []
spk_doa_cls_list = []
for idx, ex in enumerate(train_dl):
    mix_sph, spk_dp_signal, spk_doa, noi_doa = ex
    #computing speech fraction frames
    if 0:
        sig_vad = compute_vad(spk_dp_signal[0,0,:].numpy(), 320, 160, 16000)
        sph_frm_perc = torch.round(torch.mean(sig_vad*1.0)*100, decimals=1)
        #print(f"idx: {idx}, sph_frms: {sph_frm_perc:.2f} %")
        sph_frm_perc_list.append(sph_frm_perc)
    else:
        spk_doa_cls_list.append(spk_doa//5)

   

In [ ]:
import matplotlib.pyplot as plt
plt.hist(spk_doa_cls_list, 37)

In [ ]:
spk_doa_cls_list

In [ ]:
mix_sph, spk_dp_signal, spk_doa, noi_doa = train_dataset[1]

In [ ]:
sig_vad = compute_vad(spk_dp_signal[0,:].numpy(), 320, 160, 16000)

In [ ]:
torch.round(torch.mean(sig_vad*1.0)*100, decimals=1)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(sph_frm_perc_list, 11)


In [ ]:
plt.plot(sph_frm_perc_list)

In [ ]:
x = torch.rand(2,3,4)
x[0,:,:].shape
print(x[0,:,:])
print(torch.permute(x[0,:,:],[1,0]))

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

loss = nn.BCELoss()

input = torch.rand(16,37)

output = F.one_hot(torch.arange(0,16), 37)

In [ ]:
output.dtype

In [ ]:
loss(input, output.to(torch.float))

In [ ]:
_loss = 0
for i in range(16):
    for j in range(37):
        _loss += output[i,j]*torch.log(input[i,j]) + (1-output[i,j])*torch.log(1-input[i,j]) 

In [ ]:
_loss/(16*37)

In [10]:
# Test Mixtures check

from create_test_mix import *
import pandas as pd

df = pd.read_csv("test_files_list.csv")


config_file = "data_gen.yaml"
mode = "TestMeasuredRIR"
with open(config_file,"r") as f:
    exp_config = yaml.safe_load(f)

_dict = CreateTestScenario(exp_config[mode], 4).create_scenario( df.iloc[2], int(1.6*16000), 4)

In [11]:
ipd.Audio(_dict["mix_sph"][0,:].numpy(), rate=16000, autoplay=True)